In [1]:
import json
from dotenv import load_dotenv
from functools import reduce
import numpy as np
import operator
import os
import pandas as pd
import pymongo
from pymongo import MongoClient
import requests

In [2]:
load_dotenv()

True

In [211]:
paris_pg = {'type': 'Point', 'coordinates': [48.87241163769417,2.3317031342524017]}

In [13]:
melbourne_cbd = {'type': 'Point', 'coordinates': [-37.81227175879301,144.96262186884945]}

In [37]:
sao_paulo_zc = {'type': 'Point', 'coordinates': [-23.533215960575276,-46.639408758375076]}

## GEOQUERIES

In [14]:
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [15]:
transport = db.get_collection("transport")

In [16]:
gastronomy = db.get_collection("gastronomy")

In [59]:
schools = db.get_collection("schools")

In [239]:
veterinary = db.get_collection("veterinary")

In [19]:
basketball = db.get_collection("basketball")

In [ ]:
def geoquering(cathegory, distance, collection, city):
    c = f'{cathegory}'
    cond1 = {'categories.name': c}
    cond2 = {"location": {"$near": {"$geometry": city, "$minDistance": 0 ,"$maxDistance": int(distance)}}}
    
    gquery = {'$and': [cond1, cond2]}
    l = list(collection.find(gquery))
    return l

### PARIS

#### TRANSPORT

In [215]:
airports = geoquering('Airport', 20000, transport, paris_pg)

In [216]:
commuter_stations = geoquering('Train Station', 2000, transport, paris_pg)

In [217]:
airport_shuttle = geoquering('Bus Stop', 2000, transport, paris_pg)

In [218]:
airport_shuttle_ = geoquering('Transportation Service', 2000, transport, paris_pg)

#### SCHOOLS

In [220]:
schools1 = geoquering('Nursery School', 2000, schools, paris_pg)

In [221]:
schools2 = geoquering('Elementary School', 2000, schools, paris_pg)

In [222]:
schools3 = geoquering('High School', 2000, schools, paris_pg)

#### GASTRO

In [226]:
vegan = geoquering('Vegetarian / Vegan Restaurant', 2000, gastronomy, paris_pg)

In [227]:
vegan2 = geoquering('Restaurant', 2000, gastronomy, paris_pg)

In [228]:
vegan3 = geoquering('Snack Place', 2000, gastronomy, paris_pg)

### OJO ACA QUE HAY QUE CARGAR CATHEGORY EN STARBUCKS

In [229]:
starbucks = geoquering('Starbucks', 500, gastronomy, paris_pg)

#### VET

In [242]:
veterinarian = geoquering('Veterinarian', 2000, veterinary, paris_pg)

#### BASKET

In [264]:
basketball_court = geoquering('Basketball Court', 10000, basketball, paris_pg)

### MELBOURNE

#### TRANSPORT

In [215]:
airports = geoquering('Airport', 20000, transport, paris_pg)

In [216]:
commuter_stations = geoquering('Train Station', 2000, transport, paris_pg)

In [217]:
airport_shuttle = geoquering('Bus Stop', 2000, transport, paris_pg)

In [218]:
airport_shuttle_ = geoquering('Transportation Service', 2000, transport, paris_pg)

#### SCHOOLS

In [220]:
schools1 = geoquering('Nursery School', 2000, schools, paris_pg)

In [221]:
schools2 = geoquering('Elementary School', 2000, schools, paris_pg)

In [222]:
schools3 = geoquering('High School', 2000, schools, paris_pg)

#### GASTRO

In [226]:
vegan = geoquering('Vegetarian / Vegan Restaurant', 2000, gastronomy, paris_pg)

In [227]:
vegan2 = geoquering('Restaurant', 2000, gastronomy, paris_pg)

In [228]:
vegan3 = geoquering('Snack Place', 2000, gastronomy, paris_pg)

### OJO ACA QUE HAY QUE CARGAR CATHEGORY EN STARBUCKS

In [229]:
starbucks = geoquering('Starbucks', 500, gastronomy, paris_pg)

#### VET

In [242]:
veterinarian = geoquering('Veterinarian', 2000, veterinary, paris_pg)

#### BASKET

In [264]:
basketball_court = geoquering('Basketball Court', 10000, basketball, paris_pg)

### SAO PAULO

#### TRANSPORT

In [215]:
airports = geoquering('Airport', 20000, transport, paris_pg)

In [216]:
commuter_stations = geoquering('Train Station', 2000, transport, paris_pg)

In [217]:
airport_shuttle = geoquering('Bus Stop', 2000, transport, paris_pg)

In [218]:
airport_shuttle_ = geoquering('Transportation Service', 2000, transport, paris_pg)

#### SCHOOLS

In [220]:
schools1 = geoquering('Nursery School', 2000, schools, paris_pg)

In [221]:
schools2 = geoquering('Elementary School', 2000, schools, paris_pg)

In [222]:
schools3 = geoquering('High School', 2000, schools, paris_pg)

#### GASTRO

In [226]:
vegan = geoquering('Vegetarian / Vegan Restaurant', 2000, gastronomy, paris_pg)

In [227]:
vegan2 = geoquering('Restaurant', 2000, gastronomy, paris_pg)

In [228]:
vegan3 = geoquering('Snack Place', 2000, gastronomy, paris_pg)

### OJO ACA QUE HAY QUE CARGAR CATHEGORY EN STARBUCKS

In [229]:
starbucks = geoquering('Starbucks', 500, gastronomy, paris_pg)

#### VET

In [242]:
veterinarian = geoquering('Veterinarian', 2000, veterinary, paris_pg)

#### BASKET

In [264]:
basketball_court = geoquering('Basketball Court', 10000, basketball, paris_pg)

## GEO DESICIÓN

In [305]:
geoqueries_paris = np.array([len(airports), len(commuter_stations), len(airport_shuttle),
                    (len(schools1) + len(schools2) + len(schools3)), (len(vegan)+ len(vegan2) + len(vegan3)),
                    len(veterinarian), len(starbucks), len(basketball_court)])

In [ ]:
geoqueries_melbourne = np.array([len(airports), len(commuter_stations), len(airport_shuttle),
                    (len(schools1) + len(schools2) + len(schools3)), (len(vegan)+ len(vegan2) + len(vegan3)),
                    len(veterinarian), len(starbucks), len(basketball_court)])

In [ ]:
geoqueries_saopaulo = np.array([len(airports), len(commuter_stations), len(airport_shuttle),
                    (len(schools1) + len(schools2) + len(schools3)), (len(vegan)+ len(vegan2) + len(vegan3)),
                    len(veterinarian), len(starbucks), len(basketball_court)])

In [297]:
weights = np.array([0.19, 0.18, 0.17, 0.14, 0.12, 0.1, 0.05, 0.05])

In [306]:
score_p = {'Conditions': ['Airports', 'Commuter Station', 'Airport Shuttle', 'Schools', 'Vegan Restaurants', 'Veterinaries', 'Starbucks Cafe', 'Basketball Court'],
           'Weights': [0.19, 0.18, 0.17, 0.14, 0.12, 0.1, 0.05, 0.05],
           'Geoqueries Paris': geoqueries_paris, 'Score Paris': weights * geoqueries_paris}

In [307]:
score_p = pd.DataFrame(score_p)

In [308]:
score_p

,Conditions,Weights,Geoqueries Paris,Score Paris
0,Airports,0.19,2,0.38
1,Commuter Station,0.18,1,0.18
2,Airport Shuttle,0.17,3,0.51
3,Schools,0.14,8,1.12
4,Vegan Restaurants,0.12,5,0.60
5,Veterinaries,0.10,7,0.70
6,Starbucks Cafe,0.05,0,0.00
7,Basketball Court,0.05,5,0.25
